In [ ]:
# Forcing autoreload of modules so restart is not required
%load_ext autoreload
%autoreload 2

import os

os.chdir("../../")


print("Current Working Directory:  ", os.getcwd())

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [3]:
train = pd.read_csv("./data/staging/NSL_KDD/train.csv")
test = pd.read_csv("./data/staging/NSL_KDD/test.csv")

In [4]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [5]:
print(train['41'].unique())
print(test['41'].unique())

['normal' 'neptune' 'warezclient' 'ipsweep' 'portsweep' 'teardrop' 'nmap'
 'satan' 'smurf' 'pod' 'back' 'guess_passwd' 'ftp_write' 'multihop'
 'rootkit' 'buffer_overflow' 'imap' 'warezmaster' 'phf' 'land'
 'loadmodule' 'spy' 'perl']
['neptune' 'normal' 'saint' 'mscan' 'guess_passwd' 'smurf' 'apache2'
 'satan' 'buffer_overflow' 'back' 'warezmaster' 'snmpgetattack'
 'processtable' 'pod' 'httptunnel' 'nmap' 'ps' 'snmpguess' 'ipsweep'
 'mailbomb' 'portsweep' 'multihop' 'named' 'sendmail' 'loadmodule' 'xterm'
 'worm' 'teardrop' 'rootkit' 'xlock' 'perl' 'land' 'xsnoop' 'sqlattack'
 'ftp_write' 'imap' 'udpstorm' 'phf']


In [6]:
print(len(train))
print(len(test))

125973
22544


In [7]:
valid_labels = ['normal', 'neptune', 'smurf', 'teardrop', 'pod', 'land', 'back']

In [8]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [9]:
print(len(train))
print(len(test))

125973
22544


In [10]:
print(train['41'].unique())
print(test['41'].unique())

['normal' 'neptune' 'warezclient' 'ipsweep' 'portsweep' 'teardrop' 'nmap'
 'satan' 'smurf' 'pod' 'back' 'guess_passwd' 'ftp_write' 'multihop'
 'rootkit' 'buffer_overflow' 'imap' 'warezmaster' 'phf' 'land'
 'loadmodule' 'spy' 'perl']
['neptune' 'normal' 'saint' 'mscan' 'guess_passwd' 'smurf' 'apache2'
 'satan' 'buffer_overflow' 'back' 'warezmaster' 'snmpgetattack'
 'processtable' 'pod' 'httptunnel' 'nmap' 'ps' 'snmpguess' 'ipsweep'
 'mailbomb' 'portsweep' 'multihop' 'named' 'sendmail' 'loadmodule' 'xterm'
 'worm' 'teardrop' 'rootkit' 'xlock' 'perl' 'land' 'xsnoop' 'sqlattack'
 'ftp_write' 'imap' 'udpstorm' 'phf']


In [11]:
dos_attacks = ['neptune', 'smurf', 'teardrop', 'pod', 'land', 'back']

train['label'] = train['41'].map(lambda x: 0 if x == 'normal' else 1 if x in dos_attacks else 2)
test['label'] = test['41'].map(lambda x: 0 if x == 'normal' else 1 if x in dos_attacks else 2)

In [12]:
train.tail(500)

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,label
125473,0,tcp,private,S0,0,0,0,0,0,0,...,0.08,0.00,0.0,1.00,1.0,0.00,0.0,neptune,21,1
125474,0,tcp,mtp,REJ,0,0,0,0,0,0,...,0.07,0.00,0.0,0.00,0.0,1.00,1.0,neptune,20,1
125475,0,tcp,ftp,S0,0,0,0,0,0,0,...,0.06,0.00,0.0,1.00,1.0,0.00,0.0,neptune,19,1
125476,0,tcp,ldap,REJ,0,0,0,0,0,0,...,0.06,0.00,0.0,0.00,0.0,1.00,1.0,neptune,18,1
125477,0,tcp,private,REJ,0,0,0,0,0,0,...,0.99,0.98,0.0,0.00,0.0,0.98,1.0,portsweep,18,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.06,0.00,0.0,1.00,1.0,0.00,0.0,neptune,20,1
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.01,0.01,0.0,0.00,0.0,0.00,0.0,normal,21,0
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.06,0.00,0.0,0.72,0.0,0.01,0.0,normal,18,0
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.05,0.00,0.0,1.00,1.0,0.00,0.0,neptune,20,1


In [13]:
print(len(train))
print(len(test))

125973
22544


In [18]:
print(train.shape)
print(test.shape)

(125973, 44)
(22544, 44)


In [19]:
extra_cols_test = set(test.columns) - set(train.columns)
missing_cols_train = set(train.columns) - set(test.columns)

print("Columns in test but not in train:", extra_cols_test)
print("Columns in train but not in test:", missing_cols_train)


Columns in test but not in train: set()
Columns in train but not in test: set()


In [ ]:
test = test[train.columns]

In [21]:
print(train.shape)
print(test.shape)

(125973, 44)
(22544, 44)


In [22]:
extra_cols_test = set(test.columns) - set(train.columns)
missing_cols_train = set(train.columns) - set(test.columns)

print("Columns in test but not in train:", extra_cols_test)
print("Columns in train but not in test:", missing_cols_train)


Columns in test but not in train: set()
Columns in train but not in test: set()


In [23]:
X_train, Y_train = train.drop(columns=['label']), train['label']
X_test, Y_test = test.drop(columns=['label']), test['label']

In [24]:
type(Y_train)

pandas.core.series.Series

In [25]:
label_encoder_train = LabelEncoder()
Y_train = label_encoder_train.fit_transform(Y_train)


Y_test = label_encoder_train.transform(Y_test)

In [26]:
Y_train

array([0, 0, 1, ..., 0, 1, 0], shape=(125973,))

In [27]:
Y_test

array([1, 1, 0, ..., 1, 0, 2], shape=(22544,))

In [28]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [29]:
print(train.dtypes.value_counts())
print(test.dtypes.value_counts())

int64      25
float64    15
object      4
Name: count, dtype: int64
int64      25
float64    15
object      4
Name: count, dtype: int64


In [30]:
print(train.select_dtypes(exclude=np.number).columns)
print(test.select_dtypes(exclude=np.number).columns)

Index(['1', '2', '3', '41'], dtype='object')
Index(['1', '2', '3', '41'], dtype='object')


In [31]:
categories_col = ['1', '2', '3','41']
num_col_train = list(set(X_train.columns) - set(categories_col))
num_col_test = list(set(X_test.columns) - set(categories_col))

In [32]:
print(train.dtypes.value_counts())


int64      25
float64    15
object      4
Name: count, dtype: int64


In [33]:
scaler = StandardScaler()
scaler.fit(X_train[num_col_train])

StandardScaler()

In [ ]:
X_train[num_col_train] = scaler.transform(X_train[num_col_train])
X_test[num_col_test] = scaler.transform(X_test[num_col_test])

In [35]:
X_test.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42'],
      dtype='object')

In [36]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,-0.110249,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.782367,-0.280282,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,normal,0.216426
1,-0.110249,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.161030,2.736852,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal,-1.965556
2,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.938287,-0.174417,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,neptune,-0.219970
3,-0.110249,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,1.066401,-0.439078,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,normal,0.652823
4,-0.110249,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,1.066401,-0.439078,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal,0.652823


In [37]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,-0.110249,tcp,private,REJ,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.071933,-0.121485,-0.480197,-0.289103,-0.639532,-0.624871,2.874410,2.753914,neptune,0.652823
1,-0.110249,tcp,private,REJ,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.161030,-0.121485,-0.480197,-0.289103,-0.639532,-0.624871,2.874410,2.753914,neptune,0.652823
2,-0.109481,tcp,ftp_data,SF,-0.005551,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,0.197703,-0.227350,1.493939,-0.111426,-0.639532,-0.624871,-0.387635,-0.376387,normal,0.652823
3,-0.110249,icmp,eco_i,SF,-0.007759,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,1.066401,-0.439078,2.756092,2.198385,-0.639532,-0.624871,-0.387635,-0.376387,saint,-1.965556
4,-0.109865,tcp,telnet,RSTO,-0.007762,-0.004915,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.470526,0.460769,-0.383108,-0.111426,-0.639532,-0.624871,2.319862,1.846127,mscan,-3.711143


In [38]:
X_train.drop('41',axis=1,inplace=True)
X_test.drop('41',axis=1,inplace=True)

In [ ]:
service_ohe = OneHotEncoder(handle_unknown="ignore")
proto_ohe = OneHotEncoder(handle_unknown="ignore")
flag_ohe = OneHotEncoder(handle_unknown="ignore")

ohe_service_train = service_ohe.fit(X_train[['2']])
ohe_proto_train = proto_ohe.fit(X_train[['1']])
ohe_flag_train = flag_ohe.fit(X_train[['3']])

X_train_service = pd.DataFrame(ohe_service_train.transform(X_train[['2']]).todense(), 
							   columns=ohe_service_train.get_feature_names_out(['2']))
X_train_proto = pd.DataFrame(ohe_proto_train.transform(X_train[['1']]).todense(), 
							 columns=ohe_proto_train.get_feature_names_out(['1']))
X_train_state = pd.DataFrame(ohe_flag_train.transform(X_train[['3']]).todense(), 
							 columns=ohe_flag_train.get_feature_names_out(['3']))

X_train = X_train.drop(['2', '1', '3'], axis=1)
X_train = pd.concat([X_train, X_train_service, X_train_proto, X_train_state], axis=1)

X_test_service = pd.DataFrame(ohe_service_train.transform(X_test[['2']]).todense(), 
							  columns=ohe_service_train.get_feature_names_out(['2']))
X_test_proto = pd.DataFrame(ohe_proto_train.transform(X_test[['1']]).todense(), 
							columns=ohe_proto_train.get_feature_names_out(['1']))
X_test_state = pd.DataFrame(ohe_flag_train.transform(X_test[['3']]).todense(), 
							columns=ohe_flag_train.get_feature_names_out(['3']))

X_test = X_test.drop(['2', '1', '3'], axis=1)
X_test = pd.concat([X_test, X_test_service, X_test_proto, X_test_state], axis=1)

In [40]:
X_train.head()

,0,4,5,6,7,8,9,10,11,12,...,3_REJ,3_RSTO,3_RSTOS0,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,-0.110249,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.110249,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,-0.110249,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.110249,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,1.235694,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [41]:
X_test.head()

,0,4,5,6,7,8,9,10,11,12,...,3_REJ,3_RSTO,3_RSTOS0,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.110249,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.109481,-0.005551,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.110249,-0.007759,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.109865,-0.007762,-0.004915,-0.014089,-0.089486,-0.007736,-0.095076,-0.027023,-0.809262,-0.011664,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x_train = set(X_train.columns.tolist())
x_test = set(X_test.columns.tolist())

diff_train = x_train - x_test  
diff_test = x_test - x_train

print("Columns in X_train but not in X_test:", diff_train)
print("Columns in X_test but not in X_train:", diff_test)

print(X_train.shape, X_test.shape)

Columns in X_train but not in X_test: set()
Columns in X_test but not in X_train: set()
(125973, 123) (22544, 123)


In [43]:
Y_train = pd.DataFrame(Y_train, columns=['class'])
Y_test = pd.DataFrame(Y_test, columns=['class'])

In [44]:
Y_train['class'].unique()

array([0, 1, 2])

In [45]:
Y_test['class'].unique()

array([1, 0, 2])

In [ ]:
X_train = X_train.dropna()
Y_train = Y_train.loc[X_train.index]
X_test = X_test.dropna()
Y_test = Y_test.loc[X_test.index]

In [ ]:
x_train = set(X_train.columns.tolist())
x_test = set(X_test.columns.tolist())

diff_train = x_train - x_test
diff_test = x_test - x_train

print("Columns in X_train but not in X_test:", diff_train)
print("Columns in X_test but not in X_train:", diff_test)

print(X_train.shape, X_test.shape)

Columns in X_train but not in X_test: set()
Columns in X_test but not in X_train: set()
(125973, 123) (22544, 123)


In [48]:
X_train.to_csv('./data/processed/NSL_KDD/multi_class/X_train.csv', index=False)
Y_train.to_csv('./data/processed/NSL_KDD/multi_class/Y_train.csv', index=False)

In [49]:
X_test.to_csv('./data/processed/NSL_KDD/multi_class/X_test.csv', index=False)
Y_test.to_csv('./data/processed/NSL_KDD/multi_class/Y_test.csv', index=False)